# Bitbucket file generator

This is a quick script for generating a new skeleton low level FUSE file system:

* It reads `fuse_lowlevel.h` and finds the code definition(s)
* It generates files with the function prototype name (e.g., "init.c")
* It generates a stub for the function prototype
* It generates a list of the files for inclusion into the meson.build file


In [19]:
fuse_lowlevel_h = "../../include/fuse_lowlevel.h"

with open(fuse_lowlevel_h, "rt") as fd:
    fuse_ll_lines = fd.readlines()

index = 0
functions = {}
for l in fuse_ll_lines:
    # Step (1): find the start of the operations definition
    index = index + 1
    if 'struct fuse_lowlevel_ops' in l: break
while index < len(fuse_ll_lines):
    if 'void (*' in fuse_ll_lines[index]:
        func = fuse_ll_lines[index][2+fuse_ll_lines[index].find('('):fuse_ll_lines[index].find(')')]
        functions[func] = fuse_ll_lines[index].rstrip()
        while True:
            if ');' in fuse_ll_lines[index]: break
            index = index + 1
            functions[func] += fuse_ll_lines[index].rstrip()
    if fuse_ll_lines[index].startswith('};'): break
    index = index + 1
for func in functions:
    # functions[func] = functions[func].replace('\t', '')
    functions[func] = ' '.join(functions[func].split())
for func in functions:
    print(functions[func])

void (*init)(void *userdata, struct fuse_conn_info *conn);
void (*destroy)(void *userdata);
void (*lookup)(fuse_req_t req, fuse_ino_t parent, const char *name);
void (*forget)(fuse_req_t req, fuse_ino_t ino, uint64_t nlookup);
void (*getattr)(fuse_req_t req, fuse_ino_t ino, struct fuse_file_info *fi);
void (*setattr)(fuse_req_t req, fuse_ino_t ino, struct stat *attr, int to_set, struct fuse_file_info *fi);
void (*readlink)(fuse_req_t req, fuse_ino_t ino);
void (*mknod)(fuse_req_t req, fuse_ino_t parent, const char *name, mode_t mode, dev_t rdev);
void (*mkdir)(fuse_req_t req, fuse_ino_t parent, const char *name, mode_t mode);
void (*unlink)(fuse_req_t req, fuse_ino_t parent, const char *name);
void (*rmdir)(fuse_req_t req, fuse_ino_t parent, const char *name);
void (*symlink)(fuse_req_t req, const char *link, fuse_ino_t parent, const char *name);
void (*rename)(fuse_req_t req, fuse_ino_t parent, const char *name, fuse_ino_t newparent, const char *newname, unsigned int flags);
void (*li

In [20]:
for func in functions:
    cfunc = functions[func]
    proto_start = cfunc.find('(*')
    proto_end = cfunc.find(')(')
    proto_end_length = 1
    if proto_end <= proto_start:
        proto_end = cfunc.find(') (')
        proto_end_length = 2
    assert proto_end > proto_start, '{}: start = {}, end = {}'.format(func, proto_start, proto_end)
    statement_end = cfunc.find(';')
    nfunc = cfunc[:proto_start] + 'bitbucket_' + cfunc[proto_start + 2: proto_end] + cfunc[proto_end + proto_end_length:statement_end]
    nfunc = nfunc + '\n{\n\tassert(0); // Not implemented\n}\n'
    functions[func] = nfunc

# At this point we have prototype wrappers

In [21]:
print(os.getcwd())
for func in functions:
    with open('{}.c'.format(func), "wt") as fd:
        fd.write('//\n// (C) Copyright 2020\n// Tony Mason\n// All Rights Reserved\n\n')
        fd.write('#include "bitbucket.h"\n\n')
        fd.write(functions[func])

/home/mason/projects/finesse/finesse/bitbucket


In [22]:
files = ['{}.c'.format(function) for function in functions]
files.sort()
for f in files: print("\t'{}',".format(f))

'access.c',
	'bmap.c',
	'copy_file_range.c',
	'create.c',
	'destroy.c',
	'fallocate.c',
	'flock.c',
	'flush.c',
	'forget.c',
	'forget_multi.c',
	'fsync.c',
	'fsyncdir.c',
	'getattr.c',
	'getlk.c',
	'getxattr.c',
	'init.c',
	'ioctl.c',
	'link.c',
	'listxattr.c',
	'lookup.c',
	'lseek.c',
	'mkdir.c',
	'mknod.c',
	'open.c',
	'opendir.c',
	'poll.c',
	'read.c',
	'readdir.c',
	'readdirplus.c',
	'readlink.c',
	'release.c',
	'releasedir.c',
	'removexattr.c',
	'rename.c',
	'retrieve_reply.c',
	'rmdir.c',
	'setattr.c',
	'setlk.c',
	'setxattr.c',
	'statfs.c',
	'symlink.c',
	'unlink.c',
	'write.c',
	'write_buf.c',
